# Welcome to BASS!

Version: Single Wave- Interactive Notebook.

    BASS: Biomedical Analysis Software Suite for event detection and signal processing.
    Copyright (C) 2015  Abigail Dobyns

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>

## Initalize

Run the following code block to intialize the program. This notebook and bass.py file **must** be in the same folder.

In [1]:
from bass import *

BASS ready!


# Begin User Input

For help, check out the wiki: https://github.com/drcgw/bass/wiki/Tutorial

Or the video tutorial: Coming Soon!

## Load Data File

Use the following block to change your settings. You **must** use this block.

Here are some helpful information about the loading settings:

**Full File Path to Folder containing file:**
Designate the path to your file to load. It can also be the relative path to the folder where this notebook is stored. This does not include the file itself.

    Mac OSX Example: '/Users/MYNAME/Documents/bass'
    Microsoft Example: 'C:\\Users\MYNAME\Documents\bass'
    

**File name:**
This is the name of your data file. It should include the file type. This file should NOT have a header and the first column must be time in seconds. Note: This file name will also appear as part of the output files names.

    'rat34_ECG.txt' 

**Full File Path for data output:** Designate the location of the folder where you would like the folder containing your results to go. If the folder does not exist, then it will be created. A plots folder, called 'plots' will be created inside this folder for you if it does not already exist. 

    Mac OSX Example: '/Users/MYNAME/Documents/output'
    Microsoft Example: 'C:\\Users\MYNAME\Documents\output' 

https://github.com/drcgw/SWAN/wiki/Tutorial#load

In [2]:
Data, Settings, Results = load_interact()

Full File Path to Folder containing file: /Users/abigaildobyns/Desktop
File Name: rat34_ECG.txt
Full File Path to Output Folder: /Users/abigaildobyns/Desktop/demo
Made plots folder
Data Loaded


### Graph Data (Optional)

Use this block to check any slicing you need to do to cut out problematic data from the head or tail. You can click on any point in the wave to get the (x,y) location of that point. Clipping inside this notebook is not supported at this time.

https://github.com/drcgw/SWAN/wiki/Tutorial#graph-data

In [3]:
plot_rawdata(Data)

In [3]:
graph_ts(Data, Settings, Results)

### Power Spectral Density (Optional)

Use the settings code block to set your frequency bands to calculate area under the curve. This block is not required. band output is always in raw power, even if the graph scale is dB/Hz.

In [23]:
#optional
Settings['PSD-Signal'] = Series(index = ['ULF', 'VLF', 'LF','HF','dx'])
#Set PSD ranges for power in band

Settings['PSD-Signal']['ULF'] = 100 #max of the range of the ultra low freq band. range is 0:ulf
Settings['PSD-Signal']['VLF'] = 200 #max of the range of the very low freq band. range is ulf:vlf
Settings['PSD-Signal']['LF'] = 300 #max of the range of the low freq band. range is vlf:lf
Settings['PSD-Signal']['HF'] = 400 #max of the range of the high freq band. range is lf:hf. hf can be no more than (hz/2)
Settings['PSD-Signal']['dx'] = 10 #segmentation for the area under the curve. 

Use the block below to generate the PSD graph and power in bands results (if selected). scale toggles which units to use for the graph:
    
    raw = s^2/Hz
    db = dB/Hz = 10*log10(s^2/Hz)

In [6]:
scale = 'raw' #raw or db
Results = psd_signal(version = 'original', key = 'Mean1', scale = scale, 
                     Data = Data, Settings = Settings, Results = Results)
Results['PSD-Signal']

Could not calculate area in bands.


""
ULF
VLF
LF
HF
LF/HF
Scale


In [35]:
sig = Data['original']['Mean1'].tolist()
hertz = int(1/Settings['Sample Rate (s/frame)']) #must be int to protect welch
Fxx, Pxx = scipy.signal.welch(sig, fs = hertz, window="hanning", nperseg=2*hertz, noverlap=hertz/2, nfft=2*hertz, detrend='linear', return_onesided=True, scaling='density')

results_psd = Series(index = ['ULF', 'VLF', 'LF','HF','LF/HF', 'Scale'])

FXX = Series(Fxx)
PXX = Series(Pxx)
PXX

0       1.089407e-07
1       1.850597e-07
2       6.456725e-09
3       4.587023e-10
4       1.089729e-10
5       5.553860e-11
6       4.728741e-11
7       4.656378e-11
8       4.199157e-11
9       1.258173e-10
10      9.263317e-10
11      1.050929e-09
12      2.382794e-10
13      4.672270e-11
14      4.249299e-11
15      4.397005e-11
16      4.203848e-11
17      4.469125e-11
18      4.757747e-11
19      6.598750e-11
20      2.109989e-10
21      2.725922e-10
22      1.608454e-10
23      7.201972e-11
24      3.967659e-11
25      4.118384e-11
26      4.417218e-11
27      4.218644e-11
28      4.084873e-11
29      4.486987e-11
            ...     
3971    4.377918e-11
3972    4.229905e-11
3973    4.247855e-11
3974    4.102055e-11
3975    3.949615e-11
3976    3.893065e-11
3977    4.088004e-11
3978    4.291905e-11
3979    3.941524e-11
3980    3.932730e-11
3981    4.020895e-11
3982    4.068548e-11
3983    4.099701e-11
3984    4.018573e-11
3985    4.066344e-11
3986    4.351346e-11
3987    4.418

In [44]:
df_power = DataFrame( index = Fxx, data = Pxx)
df_power.columns = ['Power']
df_power

,Power
0.0,1.089407e-07
0.5,1.850597e-07
1.0,6.456725e-09
1.5,4.587023e-10
2.0,1.089729e-10
2.5,5.553860e-11
3.0,4.728741e-11
3.5,4.656378e-11
4.0,4.199157e-11
4.5,1.258173e-10


In [46]:
df_ulf = df_power[df_power.index<Settings['PSD-Signal']['ULF']]
df_ulf.index.tolist()

[0.0,
 0.5,
 1.0,
 1.5,
 2.0,
 2.5,
 3.0,
 3.5,
 4.0,
 4.5,
 5.0,
 5.5,
 6.0,
 6.5,
 7.0,
 7.5,
 8.0,
 8.5,
 9.0,
 9.5,
 10.0,
 10.5,
 11.0,
 11.5,
 12.0,
 12.5,
 13.0,
 13.5,
 14.0,
 14.5,
 15.0,
 15.5,
 16.0,
 16.5,
 17.0,
 17.5,
 18.0,
 18.5,
 19.0,
 19.5,
 20.0,
 20.5,
 21.0,
 21.5,
 22.0,
 22.5,
 23.0,
 23.5,
 24.0,
 24.5,
 25.0,
 25.5,
 26.0,
 26.5,
 27.0,
 27.5,
 28.0,
 28.5,
 29.0,
 29.5,
 30.0,
 30.5,
 31.0,
 31.5,
 32.0,
 32.5,
 33.0,
 33.5,
 34.0,
 34.5,
 35.0,
 35.5,
 36.0,
 36.5,
 37.0,
 37.5,
 38.0,
 38.5,
 39.0,
 39.5,
 40.0,
 40.5,
 41.0,
 41.5,
 42.0,
 42.5,
 43.0,
 43.5,
 44.0,
 44.5,
 45.0,
 45.5,
 46.0,
 46.5,
 47.0,
 47.5,
 48.0,
 48.5,
 49.0,
 49.5,
 50.0,
 50.5,
 51.0,
 51.5,
 52.0,
 52.5,
 53.0,
 53.5,
 54.0,
 54.5,
 55.0,
 55.5,
 56.0,
 56.5,
 57.0,
 57.5,
 58.0,
 58.5,
 59.0,
 59.5,
 60.0,
 60.5,
 61.0,
 61.5,
 62.0,
 62.5,
 63.0,
 63.5,
 64.0,
 64.5,
 65.0,
 65.5,
 66.0,
 66.5,
 67.0,
 67.5,
 68.0,
 68.5,
 69.0,
 69.5,
 70.0,
 70.5,
 71.0,
 71.5,
 72.0,
 72.5

In [49]:
scipy.integrate.simps(df_ulf['Power'], df_ulf.index, dx =)

0.026821989570022672

In [ ]:
results_psd['ULF'] = scipy.integrate.simps(PXX[FXX<Settings['PSD-Signal']['ULF']].tolist(), 
                           FXX[FXX<Settings['PSD-Signal']['ULF']].tolist(), 
                           dx =Settings['PSD-Signal']['dx'])
results_psd['VLF'] = scipy.integrate.simps(PXX[(Settings['PSD-Signal']['ULF']<FXX) & (FXX<=Settings['PSD-Signal']['VLF'])].tolist(),
                           FXX[(Settings['PSD-Signal']['ULF']<FXX) & (FXX<=Settings['PSD-Signal']['VLF'])].tolist(), 
                           dx= Settings['PSD-Signal']['dx'])
results_psd['LF'] = scipy.integrate.simps(PXX[(Settings['PSD-Signal']['VLF']<FXX) & (FXX<=Settings['PSD-Signal']['LF'])].tolist(),
                          FXX[(Settings['PSD-Signal']['VLF']<FXX) & (FXX<=Settings['PSD-Signal']['LF'])].tolist(), 
                          dx= Settings['PSD-Signal']['dx'])
results_psd['HF'] = scipy.integrate.simps(PXX[(Settings['PSD-Signal']['LF']<FXX) & (FXX<=Settings['PSD-Signal']['HF'])].tolist(),
                          FXX[(Settings['PSD-Signal']['LF']<FXX) & (FXX<=Settings['PSD-Signal']['HF'])].tolist(), 
                          dx= Settings['PSD-Signal']['dx'])
results_psd['LF/HF'] = results_psd['LF']/results_psd['HF']

results_psd

### Spectrogram (Optional)

Use the block below to get the spectrogram of the signal. The frequency (y-axis) scales automatically to only show 'active' frequencies. This can take some time to run.

    version = 'original'
    key = 'Mean1'

In [ ]:
version = 'original'
key = 'Mean1'
spectogram(version, key, Data, Settings, Results)

## Transforming Data

Must be done for each new uploaded data file.

**WARNING:** If you do not load a settings file OR enter your own settings, the analysis will not run. There are no defaults. This section is not optional.

https://github.com/drcgw/SWAN/wiki/Tutorial#transform-data

### Load settings from a file

Must be a previously outputed BASS settings file, although the name can be changed. Expected format is '.csv'. Enter the full file path and name.

    Mac OSX Example: '/Users/MYNAME/Documents/rat34_Settings.csv'
    Microsoft Example: 'C:\\Users\MYNAME\Documents\rat34_Settings.csv'
    
See above instructions for how to load your data file.

**Warning!!** You must load a settings file or specify your settings below. There are no defaults

In [27]:
Settings = load_settings_interact(Settings)
Settings_display = display_settings(Settings)
Settings_display

Full File path and file for the settings file: /Users/abigaildobyns/Desktop/rat34_Settings.csv


,Value
Absolute Value,True
Bandpass Highcut,none
Bandpass Lowcut,none
Bandpass Polynomial,none
Baseline Start,0
Baseline Stop,5
Baseline Type,static
Burst Area,False
Delta,0.25
Exclude Edges,True


### Enter your settings for data transformation

**WARNING:** If you do not load a settings file OR enter your own settings, the analysis will not run. There are no defaults. This section is not optional.

Enter the parameters of the functions you would like to use to transform your data. 
If you do not want to use a function, enter 'none'

For more help on settings:

https://github.com/drcgw/SWAN/wiki/Tutorial#transform-data

https://github.com/drcgw/SWAN/wiki/Transformation-Settings

In [4]:
Settings = user_input_trans(Settings)

Enter True or False to turn on or off the linear fit.
Linear Fit (True/False): False
Enter the butterworth bandpass settings seperated by a comma. cuts are in hertz and poly should be an interger.
Bandpass Settings (lowcut, highcut,polynomial): none
Enter the Savitzky Golay filter settings seperated by a comma. Window size must be odd.
Savitzky Golay Settings (window, polynomial): 251, 4
Settings Saved


### Run data transformation

In [28]:
Data, Settings = transform_wrapper(Data, Settings)
graph_ts(Data, Settings, Results)

## Set Baseline for Thresholding

**WARNING** If you do not load a settings file OR enter your own settings, the analysis will not run. There are no defaults. This section is not optional.

https://github.com/drcgw/SWAN/wiki/Tutorial#set-baseline-for-thresholding

### Choose either linear or rolling baseline.

**Linear** - assumes that the first 0.1 seconds is a good representation of baseline. If the superstructure is linear but has a slope, use linear fit in the transformation to still use linear. Linear automatically shifts your data by the ammount of your baseline

**Static** - Allows you to choose an arbitrary y value for threshold. No Shift in the data.

**Rolling** - rolling mean using a window. User provides the window size in miliseconds. there is no shift in the data with this method.

In [13]:
Settings = user_input_base(Settings)

Previous Baseline Type: rolling
Enter Linear, Rolling, or Static: static


### Run baseline

In [29]:
Data, Settings, Results = baseline_wrapper(Data, Settings, Results)
graph_ts(Data, Settings, Results)

## Display Settings (Optional)

Optional block. Run this at any time to check what your settings are. If it does not appear, it has not been set yet.

https://github.com/drcgw/SWAN/wiki/Tutorial#display-settings

In [ ]:
Settings_display = display_settings(Settings)
Settings_display

# Event Detection

## Peaks

https://github.com/drcgw/SWAN/wiki/Tutorial#peaks

### Enter Delta value

Choose a delta value (if not already specified) for peak detection. 
Delta is a 'top-down' threshold for peaks, where a point is considered a peak if there are points lower that it by delta.
A larger delta will result in fewer, more selective peaks.

In [15]:
Settings = event_peakdet_settings(Data, Settings)

Enter delta value between 0 and 0.698: 0.2
Enter Peak Minimum value between -0.0642 and 0.6337: -1
Enter Peak Maximum value between -1.0 and 0.6337: 1


### Run Event Peak Detection

Run block of code below to run peak deteaction. This block will display a plot of the detected events. It will also print a summary table of the peak amplitude and RR intervals.

In [30]:
Results = event_peakdet_wrapper(Data, Settings, Results)
Results['Peaks-Master'].groupby(level=0).describe()

Peaks Amplitude    Intervals
Mean1 count      3224.000000  3223.000000
      mean          0.514326     0.188253
      std           0.047384     0.007427
      min           0.403260     0.170000
      25%           0.468488     0.183250
      50%           0.526320     0.188500
      75%           0.550066     0.194250
      max           0.633748     0.202500

### Plot Events (Optional)

Call a column of data by its key (column name). Default name for one column of data is 'Mean1'

In [17]:
roi = 'Mean1'
graph_ts(Data, Settings, Results, roi)

## Bursts

https://github.com/drcgw/SWAN/wiki/Tutorial#bursts

### Enter Threshold and Cluster time values

Change the threshperc to adjust the threshold line relative to baseline and cluster_time to adjust how close together bursts can be before the are considered the same event. This is optional if the settings are already set.

In [25]:
Settings = event_burstdet_settings(Data, Settings)

Previous threshold value: 0.15
Enter a threshold value between -0.0642 and 0.6337: 0.13
Previous interval value: 0.01
Enter the minimum inter-event interval in seconds:0.01
Previous Minimum Burst Duration value: 2.0
Enter the minimum burst duration in seconds:0
Previous Maximum Burst Duration value: 1.0
Enter the maximum burst duration in seconds:5
Previous Minimum Peaks per Burst number: 1
Enter the minimum number of peaks per burst:1
Previous Burst Area setting: True
Do you want to calculate Burst Area? (True/False):Flase
Previous Exclude Edges setting: True
Do you want to Exclude Edges? (True/False):False


### Run Event Burst Detection

Run block of code below to run burst deteaction. 

This block will display a plot of the detected events. 

It will also print a summary table of start and end times for each event, event duration, interburst interval, and total cycle time.

In [31]:
Results = event_burstdet_wrapper(Data, Settings, Results)
Results['Bursts-Master'].groupby(level=0).describe()

Burst Start    Burst End  Burst Duration Edge Event  \
Mean1 count  3224.000000  3224.000000     3224.000000       3224   
      mean    297.234606   297.264828        0.030222          0   
      std     176.070115   176.070941        0.001450          0   
      min       0.120250     0.148250        0.026500      False   
      25%     143.869000   143.897875        0.029250          0   
      50%     293.386625   293.416750        0.030250          0   
      75%     449.120875   449.153063        0.031250          0   
      max     606.860500   606.893000        0.033500      False   

             Interburst Interval  Total Cycle Time  Peaks per Burst  \
Mean1 count          3223.000000       3223.000000             3224   
      mean              0.158032          0.188253                1   
      std               0.006746          0.007453                0   
      min               0.142000          0.170000                1   
      25%               0.154000          0.183000                1   
      50%               0.157750          0.188500                1   
      75%               0.162750          0.194250                1   
      max               0.171750          0.202000                1   

                Peak Amp    Peak Time       Attack        Decay  \
Mean1 count  3224.000000  3224.000000  3224.000000  3224.000000   
      mean      0.514326   297.249564     0.014957     0.015265   
      std       0.047384   176.070538     0.000890     0.000722   
      min       0.403260     0.134250     0.013250     0.013250   
      25%       0.468488   143.883188     0.014250     0.014750   
      50%       0.526320   293.401375     0.014750     0.015250   
      75%       0.550066   449.137187     0.015500     0.015750   
      max       0.633748   606.875000     0.017250     0.018000   

             Intraburst Frequency  
Mean1 count           3224.000000  
      mean              33.165306  
      std                1.604423  
      min               29.850746  
      25%               32.000000  
      50%               33.057851  
      75%               34.188034  
      max               37.735849

### Plot Events (Optional)

Call a column of data by its key (column name). Default name for one column of data is 'Mean1'

In [32]:
roi = 'Mean1'
graph_ts(Data, Settings, Results, roi)

## Save all files and settings

https://github.com/drcgw/SWAN/wiki/Tutorial#save-all-files-and-settings

In [ ]:
Save_Results(Data, Settings, Results)

# Event Analysis

Now that events are detected, you can analyze them using any of the optional blocks below. 

# Display Tables

### Display Summary Results for Peaks

In [ ]:
#grouped summary for peaks
Results['Peaks-Master'].groupby(level=0).describe()

### Display Summary Results for Peaks

In [ ]:
#grouped summary for bursts
Results['Bursts-Master'].groupby(level=0).describe()

## Results Plots

### Poincare Plots

Create a Poincare Plot of your favorite varible. Choose an event type (Peaks or Bursts), measurement type, and key. Make sure that whichever varible you chose is only composed of numbers (No NaN or missing values). Calling meas = 'All' not currently supported.

Plots are saved by clicking the save button in the pop-up window with your graph. SD1 and SD2 values are not saved automatically.

Example:

    event_type = 'Bursts'
    meas = 'Peaks per Burst'
    key = 'Mean1'

https://github.com/drcgw/SWAN/wiki/Tutorial#poincare-plots

In [ ]:
#quick
event_type = 'Bursts'
meas = 'Burst Duration'
key = 'Mean1'
poincare_plot(Results[event_type][key][meas])

### Line Plots

Create line plots of the raw data as well as the data analysis. 

Plots are saved by clicking the save button in the pop-up window with your graph.

    key = 'Mean1'
    start =100 
    end= 101

https://github.com/drcgw/SWAN/wiki/Tutorial#line-plots

In [ ]:
key = 'Mean1'
start =100 #start time in seconds
end= 101#end time in seconds
results_timeseries_plot(key, start, end, Data, Settings, Results)

### Autocorrelation Plot

Display the Autocorrelation plot of your transformed data.

Choose the start and end time in seconds. May be slow

    key = 'Mean1'
    start = 0 
    end = 10

In [ ]:
#autocorrelation
key = 'Mean1'
start = 0 #seconds, where you want the slice to begin
end = 10 #seconds, where you want the slice to end. The absolute end is -1
autocorrelation_plot(Data['trans'][key][start:end])
plt.show()

### Frequency Plot

Use this block to plot changes of any measurement over time. Does not support 'all'. Example:

    event_type = 'Peaks'
    meas = 'Intervals'
    key = 'Mean1'
https://github.com/drcgw/SWAN/wiki/Tutorial#frequency-plot

In [ ]:
event_type = 'Peaks'
meas = 'Intervals'
key = 'Mean1' #'Mean1' default for single wave
frequency_plot(event_type, meas, key, Data, Settings, Results)

# Power Spectral Density

The following blocks allows you to asses the power of event measuments in the frequency domain. While you can call this block on any event measurement, it is intended to be used on interval data (or at least data with units in seconds). Reccomended:

    event_type = 'Bursts'
    meas = 'Total Cycle Time'
    key = 'Mean1'
    
    event_type = 'Peaks'
    meas = 'Intervals'
    key = 'Mean1'

Because this data is in the frequency domain, we must interpolate it in order to perform a FFT on it.
https://github.com/drcgw/SWAN/wiki/Tutorial#power-spectral-density

## Settings

Use the code block below to specify your settings for event measurment PSD.

In [33]:
Settings['PSD-Event'] = Series(index = ['Hz','ULF', 'VLF', 'LF','HF','dx'])
#Set PSD ranges for power in band

Settings['PSD-Event']['hz'] = 4.0 #freqency that the interpolation and PSD are performed with.
Settings['PSD-Event']['ULF'] = 0.03 #max of the range of the ultra low freq band. range is 0:ulf
Settings['PSD-Event']['VLF'] = 0.05 #max of the range of the very low freq band. range is ulf:vlf
Settings['PSD-Event']['LF'] = 0.15 #max of the range of the low freq band. range is vlf:lf
Settings['PSD-Event']['HF'] = 0.4 #max of the range of the high freq band. range is lf:hf. hf can be no more than (hz/2)
Settings['PSD-Event']['dx'] = 10 #segmentation for the area under the curve. 

## Event PSD

Use block below to return the PSD plot, as well as the power in the bands defined by the settings above.

In [34]:
event_type = 'Peaks'
meas = 'Intervals'
key = 'Mean1'
scale = 'raw'
Results = psd_event(event_type, meas, key, scale, Data, Settings, Results)
Results['PSD-Event'][key]

Could not calculate power in band.


""
ULF
VLF
LF
HF
LF/HF
Scale


## Analyze Events by Measurement

Generates a line plot with error bars for a given event measurement. X axis is the names of each time series. Display Only. Intended for more than one column of data.

    event_type = 'peaks'
    meas = 'Peaks Amplitude'

In [ ]:
#Get average plots, display only
event_type = 'peaks'
meas = 'Peaks Amplitude'
average_measurement_plot(event_type, meas,Results)

## Moving/Sliding Averages, Standard Deviation, and Count

Generates the moving **mean**, **standard deviation**, and **count** for a given measurement across all columns of the Data in the form of a DataFrame (displayed as a table).
Saves out the dataframes of these three results automatically with the window size in the name as a .csv.
If **meas == 'All'**, then the function will loop and produce these tables for all measurements.

    event_type = 'Peaks'
    meas = 'all'
    window = 30

In [ ]:
#Moving Stats
event_type = 'Peaks'
meas = 'all'
window = 30 #seconds
Results = moving_statistics(event_type, meas, window, Data, Settings, Results)

# HistEntropy

Calculates the histogram entropy of a measurement for each column of data. Also saves the histogram of each. If meas is set to 'all', then all available measurements from the event_type chosen will be calculated iteratevely. 

If all of the samples fall in one bin regardless of the bin size, it means we have the most predictable sitution and the entropy is 0. If we have uniformly dist function, the max entropy will be 1

    event_type = 'Bursts'
    meas = 'all'

https://github.com/drcgw/SWAN/wiki/Tutorial#histentropy

In [ ]:
#Histogram Entropy
event_type = 'Bursts'
meas = 'all'
Results = histent_wrapper(event_type, meas, Data, Settings, Results)
Results['Histogram Entropy']

# STOP HERE

You can run another file be going back the the **Begin User Input** section and chose another file path.

# Advanced user options

## Approximate entropy

this only runs if you have pyEEG installed. **WARNING: THIS FUNCTION RUNS SLOWLY**

run the below code to get the approximate entropy of any measurement. Returns the entropy of the entire results array (no windowing). I am using the following M and R values:

    M = 2  
    R = 0.2*std(measurement)
    
these values can be modified in the source code. alternatively, you can call ap_entropy directly. 

**Interpretation**: A time series containing many repetitive patterns has a relatively small ApEn; a less predictable process has a higher ApEn.

https://github.com/drcgw/SWAN/wiki/Tutorial#approximate-entropy

### Events

In [ ]:
#Approximate Entropy
event_type = 'Peaks'
meas = 'all'
Results = ap_entropy_wrapper(event_type, meas, Data, Settings, Results)
Results['Approximate Entropy']

### Time Series

In [ ]:
#Approximate Entropy on raw signal
#takes a VERY long time
from pyeeg import ap_entropy

version = 'original' #original, trans, shift, or rolling
key = 'Mean1' #Mean1 default key for one time series
start = 0 #seconds, where you want the slice to begin
end = 1 #seconds, where you want the slice to end. The absolute end is -1

ap_entropy(Data[version][key][start:end].tolist(), 2, (0.2*np.std(Data[version][key][start:end])))

### Sample Entropy

this only runs if you have pyEEG installed. **WARNING: THIS FUNCTION RUNS SLOWLY**

run the below code to get the sample entropy of any measurement. Returns the entropy of the entire results array (no windowing). I am using the following M and R values:

    M = 2  
    R = 0.2*std(measurement)
    
these values can be modified in the source code. alternatively, you can call samp_entropy directly. 

### Events

### Time Series

In [ ]:
#Sample Entropy on raw signal
#takes a VERY long time
from pyeeg import samp_entropy

version = 'original' #original, trans, shift, or rolling
key = 'Mean1' #Mean1 default key for one time series
start = 0 #seconds, where you want the slice to begin
end = 1 #seconds, where you want the slice to end. The absolute end is -1

samp_entropy(Data[version][key][start:end].tolist(), 2, (0.2*np.std(Data[version][key][start:end])))

In [ ]:
#on raw signal
#takes a VERY long time
version = 'original' #original, trans, shift, or rolling
key = 'Mean1' #Mean1 default key for one time series
start = 0 #seconds, where you want the slice to begin
end = 1 #seconds, where you want the slice to end. The absolute end is -1

samp_entropy(Data[version][key][start:end].tolist(), 2, (0.2*np.std(Data[version][key][start:end])))

# Blank Code block

you're still here, reading? you must be a dedicated super user!

If that is the case, then you must know how to code in Python. Use this space to get crazy with your own advanced analysis and stuff.